In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import torch.optim as optim 

In [3]:

# Hype-parameters
num_epochs = 4 # how many times we are running 
batch_size = 32 # 
learning_rate = .001 # 

train_transform = transforms.Compose(
                    [
                    transforms.RandomRotation(30),
                    transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
                    transforms.ColorJitter(brightness=0.2, contrast=0.2),
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,), (0.3081,)),
                    ])

mnist = datasets.MNIST(
    root='./data', # where to store data
    train=True, # tell the code it is training data
    download=True, # download the data
    transform=train_transform # transform dataset to tensor directly (no preprocessing)
) # import data
print(len(mnist))

mnist_train, mnist_test = torch.utils.data.random_split(mnist, [0.8, .2]) # split data 80/20
print(len(mnist_train))
print(len(mnist_test))  


60000
48000
12000


In [5]:
batch_size = 32

# init dataloaders to load batches into model
train_dl  = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True) 

test_dl   = torch.utils.data.DataLoader(mnist_test, batch_size=10000, shuffle=False ) 


In [6]:
# function to measure accuracy, confusion matrix, precision, recall ,f1 (metrics to measure classification)
def print_metrics_function(y_test, y_pred):
    print('Accuracy: %.6f' % accuracy_score(y_test, y_pred))
    confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    print("Confusion Matrix:")
    print(confmat)
    print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    f1_measure = f1_score(y_true=y_test, y_pred=y_pred, average='weighted')
    print('F1-mesure: %.3f' % f1_measure)
    return f1_measure

In [7]:
class Classifier6(nn.Module):

    def __init__(self):
        super().__init__()

        self.model = nn.Sequential(
            ## Convolitional Layer 1
                nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=1), # 1 input 16 filters, padding so same dim
                nn.ReLU(), # ReLU introduce non-linearity
                nn.MaxPool2d(2, 2), #pool
 
                ## Convolutional Layer 2
                nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=1), # 16 inputs 32 filters
                nn.ReLU(),
                nn.MaxPool2d(2, 2),   
 
                ## feed forward layer w/ 1024 neurons, regular layer
                nn.Flatten(),
                nn.Linear(800, 1024),    ## see how to get 800 below on last cell
                nn.ReLU(),

                nn.Linear(1024, 10), # maps to output w/ 10 classes
                nn.LogSoftmax(dim=1)
        )
   
    def forward(self, inputs):
        return self.model(inputs)
        
def training_loop( num_epochs, model, loss_fn, opt):

    losses_list = []
    
    for epoch in range(num_epochs):
        for xb, yb in train_dl:
            
            ## print( xb.shape )   ## check this comes out as [N, 1, 28, 28]
            ## yb = torch.squeeze(yb, dim=1)
            
            y_pred = model(xb)
            loss   = loss_fn(y_pred, yb)
    
            opt.zero_grad()
            loss.backward()
            opt.step()
            
        if epoch % 1 == 0:
            print(epoch, "loss=", loss)
            losses_list.append(  loss  )
            
    return losses_list

In [8]:
model = Classifier6() # create our model

opt = torch.optim.Adam(model.parameters(), lr = learning_rate) # optimizer that does steps

loss_fn = nn.CrossEntropyLoss() # type of loss func

my_losses_list = training_loop(num_epochs, model, loss_fn, opt)

0 loss= tensor(0.1019, grad_fn=<NllLossBackward0>)
1 loss= tensor(0.0654, grad_fn=<NllLossBackward0>)
2 loss= tensor(0.0083, grad_fn=<NllLossBackward0>)
3 loss= tensor(0.0150, grad_fn=<NllLossBackward0>)


In [13]:
# test handwritten digits
from PIL import Image
import numpy as np


# Load the image, replace with custom file path
image_path = [
    "0.png",
    "1.png",
    "2.png",
    "3.png",
    "4.png",
    "5.png",
    "6.png",
    "7.png",
    "8.png",
    "9.png",
]  
for path in image_path:
    image = Image.open(path)

    # convert to greyscale & resize
    image = image.convert("L")
    image = image.resize((28,28))

    # convert to tensor
    transform = transforms.ToTensor()
    image_tr = transform(image)
    image_tr = image_tr.unsqueeze(dim=0) # add batch dimension
    image_tr = 1 - image_tr

    pred = model(image_tr)
    vals, indeces = torch.max( pred, dim=1  )
    preds = indeces
    print(preds)

tensor([8])
tensor([8])
tensor([8])
tensor([8])
tensor([8])
tensor([8])
tensor([8])
tensor([8])
tensor([8])
tensor([8])


In [10]:
from sklearn.model_selection import KFold
import torch
import torch.optim as optim
import torch.nn as nn

def k_fold_valid(model_class, train_dataloader, num_folds=5, num_epochs=10, batch_size=32):
    kfold = KFold(n_splits=num_folds, shuffle=True)
    
    # Placeholder to store results from each fold
    fold_results = []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(train_dataloader.dataset)):
        print(f"Fold {fold + 1}/{num_folds}")

        # Split the DataLoader dataset into training and validation sets for this fold
        # Create subsets for training and validation by selecting indices from the DataLoader
        train_subset = torch.utils.data.Subset(train_dataloader.dataset, train_idx)
        val_subset = torch.utils.data.Subset(train_dataloader.dataset, val_idx)

        # Create new DataLoaders for each fold
        train_loader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        
        # Initialize the model and optimizer
        model = model_class()  # Instantiate the model class
        criterion = nn.CrossEntropyLoss()  # Loss function
        optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimizer

        # Training loop
        for epoch in range(num_epochs):
            model.train()  # Set the model to training mode
            running_loss = 0.0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            avg_train_loss = running_loss / len(train_loader)
            print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}")
        
        # Validation loop
        model.eval()  # Set the model to evaluation mode
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = correct / total
        print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

        fold_results.append(val_accuracy)

    # Calculate and display the average validation accuracy
    avg_accuracy = sum(fold_results) / num_folds
    print(f"Average Validation Accuracy over {num_folds} folds: {avg_accuracy * 100:.2f}%")

k_fold_valid(
    model_class = Classifier6, 
    train_dataloader = mnist_train,
    num_folds = 5,
    num_epochs = 10,
    batch_size = batch_size
)


Fold 1/5


KeyboardInterrupt: 

Fold 1/5
Epoch 1/10, Train Loss: 0.1478
Epoch 2/10, Train Loss: 0.0458
Epoch 3/10, Train Loss: 0.0306
Epoch 4/10, Train Loss: 0.0226
Epoch 5/10, Train Loss: 0.0172
Epoch 6/10, Train Loss: 0.0147
Epoch 7/10, Train Loss: 0.0110
Epoch 8/10, Train Loss: 0.0099
Epoch 9/10, Train Loss: 0.0080
Epoch 10/10, Train Loss: 0.0079
Validation Accuracy: 98.69%
Fold 2/5
Epoch 1/10, Train Loss: 0.1468
Epoch 2/10, Train Loss: 0.0456
Epoch 3/10, Train Loss: 0.0303
Epoch 4/10, Train Loss: 0.0241
Epoch 5/10, Train Loss: 0.0176
Epoch 6/10, Train Loss: 0.0164
Epoch 7/10, Train Loss: 0.0116
Epoch 8/10, Train Loss: 0.0110
Epoch 9/10, Train Loss: 0.0097
Epoch 10/10, Train Loss: 0.0109
Validation Accuracy: 98.97%
Fold 3/5
Epoch 1/10, Train Loss: 0.1421
Epoch 2/10, Train Loss: 0.0443
Epoch 3/10, Train Loss: 0.0296
Epoch 4/10, Train Loss: 0.0215
Epoch 5/10, Train Loss: 0.0171
Epoch 6/10, Train Loss: 0.0137
Epoch 7/10, Train Loss: 0.0112
Epoch 8/10, Train Loss: 0.0097
Epoch 9/10, Train Loss: 0.0091
Epoch 10/10, Train Loss: 0.0074
Validation Accuracy: 98.88%
Fold 4/5
Epoch 1/10, Train Loss: 0.1438
Epoch 2/10, Train Loss: 0.0453
Epoch 3/10, Train Loss: 0.0308
Epoch 4/10, Train Loss: 0.0228
Epoch 5/10, Train Loss: 0.0169
Epoch 6/10, Train Loss: 0.0141
Epoch 7/10, Train Loss: 0.0123
Epoch 8/10, Train Loss: 0.0105
Epoch 9/10, Train Loss: 0.0100
Epoch 10/10, Train Loss: 0.0073
Validation Accuracy: 98.92%
Fold 5/5
Epoch 1/10, Train Loss: 0.1413
Epoch 2/10, Train Loss: 0.0474
Epoch 3/10, Train Loss: 0.0307
Epoch 4/10, Train Loss: 0.0223
Epoch 5/10, Train Loss: 0.0177
Epoch 6/10, Train Loss: 0.0142
Epoch 7/10, Train Loss: 0.0105
Epoch 8/10, Train Loss: 0.0112
Epoch 9/10, Train Loss: 0.0076
Epoch 10/10, Train Loss: 0.0086
Validation Accuracy: 98.46%
Average Validation Accuracy over 5 folds: 98.78%

In [14]:
with torch.no_grad(): # detach grad tracking for tensors
    for x_real, y_real in test_dl:
        y_pred = model(  x_real  )
        
        vals, indeces = torch.max( y_pred, dim=1  )
        preds = indeces
        print_metrics_function(y_real, preds)

Accuracy: 0.971600
Confusion Matrix:
[[1007    0    2    1    0    0    4    0    3    1]
 [   0 1084    3    3    1    0    0    7   11    1]
 [   1    4  956    3    1    0    0    8   16    2]
 [   1    0    4  975    1    6    3    2   11    2]
 [   0    0    8    0  954    0    3    3   12   22]
 [   0    0    0    4    2  918   10    0   10    3]
 [   4    1    0    0    1    8  937    0    5    0]
 [   1    3   10    4    0    0    0  962    2    6]
 [   2    0    4    0    1    2    6    2  967    4]
 [   3    0    2    3    6    3    1    5   16  956]]
Precision: 0.972
Recall: 0.972
F1-mesure: 0.972
Accuracy: 0.968000
Confusion Matrix:
[[191   0   1   0   0   0   0   0   2   0]
 [  0 227   1   1   1   0   0   1   0   0]
 [  1   0 189   0   1   0   0   4   0   1]
 [  0   0   1 187   0   0   0   0   2   0]
 [  1   0   0   0 185   0   1   1   3   9]
 [  0   0   0   2   0 179   1   0   3   2]
 [  1   0   0   0   0   1 194   0   1   1]
 [  0   0   6   1   0   0   0 200   0   0]
 [ 

In [ ]:
# save model
torch.save(model.state_dict(), "team6_final_weights.pth")   

In [ ]:
# test handwritten digits
from PIL import Image
import numpy as np


# Load the image, replace with custom file path
image_path = [
    "0.png",
    "1.png",
    "2.png",
    "3.png",
    "4.png",
    "5.png",
    "6.png",
    "7.png",
    "8.png",
    "9.png",
]  
for path in image_path:
    image = Image.open(path)

    # convert to greyscale & resize
    image = image.convert("L")
    image = image.resize((28,28))

    # convert to tensor
    transform = transforms.ToTensor()
    image_tr = transform(image)
    image_tr = image_tr.unsqueeze(dim=0) # add batch dimension
    image_tr = 1 - image_tr

    pred = model(image_tr)
    vals, indeces = torch.max( pred, dim=1  )
    preds = indeces
    print(preds)

tensor([8])
tensor([1])
tensor([8])
tensor([3])
tensor([4])
tensor([9])
tensor([8])
tensor([8])
tensor([8])
tensor([8])


In [ ]:
# commented out, took out validation idk if we need

#from torch.utils.data import DataLoader
#validation_loader = DataLoader(mnist_valid, batch_size=64, shuffle=False)

# Iterate through the DataLoader to validate
#for X_valid, y_valid in validation_loader:
    # Use X_valid and y_valid for validation here
    #break  # Remove this if processing the entire validation set
#y_valid_pred = model(X_valid)  # Predictions on validation set

#val_accuracy = accuracy_score(y_valid, y_valid_pred)
#print(f"Validation Accuracy: {val_accuracy:.2f}")